In [89]:
import numpy as np
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization

In [90]:
data = pd.read_csv('simulated_nz_ocean_current_with_regions.csv')
data.head()

,SurveyId,Region,SurveyDate,SurveyAreaLatitudeStart,SurveyAreaLongitudeStart,Rubbish_Count,count_same,date average,Current_Speed,Current_Direction,Water_Temperature
0,1,Upper West Coast,05/22/2023,-37.552290,174.371686,114.507119,112,3.370723,0.549671,150.267961,17.013002
1,2,East Coast,01/24/2018,-40.802044,170.064949,133.914511,112,4.979226,0.486174,79.958812,17.453534
2,3,Upper East Coast,09/20/2020,-42.327318,170.038232,239.062053,112,2.805245,0.564769,43.151532,17.735343
3,4,Upper East Coast,07/03/2023,-45.374311,167.444834,169.757200,112,2.379577,0.652303,121.541462,20.720169
4,5,West Coast,02/23/2018,-39.411788,175.926444,38.818991,112,1.506050,0.476585,339.447493,14.625667


In [91]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 11 columns):
 #   Column                    Non-Null Count  Dtype  
---  ------                    --------------  -----  
 0   SurveyId                  100 non-null    int64  
 1   Region                    100 non-null    object 
 2   SurveyDate                100 non-null    object 
 3   SurveyAreaLatitudeStart   100 non-null    float64
 4   SurveyAreaLongitudeStart  100 non-null    float64
 5   Rubbish_Count             100 non-null    float64
 6   count_same                100 non-null    int64  
 7   date average              100 non-null    float64
 8   Current_Speed             100 non-null    float64
 9   Current_Direction         100 non-null    float64
 10  Water_Temperature         100 non-null    float64
dtypes: float64(7), int64(2), object(2)
memory usage: 8.7+ KB


In [92]:
#encode the behaviour column
le = LabelEncoder()
data['Region'] = le.fit_transform(data['Region'])

In [93]:
# Drop the 'SurveyId' and 'SurveyDate' columns
data = data.drop(columns=['SurveyId', 'SurveyDate', 'date average', 'count_same', 'SurveyAreaLatitudeStart', 'SurveyAreaLongitudeStart'])
X = data.drop('Rubbish_Count', axis=1)
y = data['Rubbish_Count']


In [94]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100 entries, 0 to 99
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Region             100 non-null    int32  
 1   Current_Speed      100 non-null    float64
 2   Current_Direction  100 non-null    float64
 3   Water_Temperature  100 non-null    float64
dtypes: float64(3), int32(1)
memory usage: 2.9 KB


In [95]:
y.info()

<class 'pandas.core.series.Series'>
RangeIndex: 100 entries, 0 to 99
Series name: Rubbish_Count
Non-Null Count  Dtype  
--------------  -----  
100 non-null    float64
dtypes: float64(1)
memory usage: 928.0 bytes


In [96]:
xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.15, random_state=42)
# Scale the data
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(xtrain)
X_test_scaled = scaler.transform(xtest)

In [98]:
model = Sequential()
model.add(Dense(256, input_dim=X_train_scaled.shape[1], activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(128, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(32, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(1, activation='linear'))

model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_squared_error'])

model.fit(X_train_scaled, ytrain, epochs=2000, batch_size=32, validation_split=0.2)

model.evaluate(X_test_scaled, ytest)



Epoch 1/2000
3/3 [==============================] - 1s 107ms/step - loss: 14379.3604 - mean_squared_error: 14379.3604 - val_loss: 20200.8633 - val_mean_squared_error: 20200.8633
Epoch 2/2000
3/3 [==============================] - 0s 22ms/step - loss: 14291.6807 - mean_squared_error: 14291.6807 - val_loss: 20199.9883 - val_mean_squared_error: 20199.9883
Epoch 3/2000
3/3 [==============================] - 0s 22ms/step - loss: 14235.7012 - mean_squared_error: 14235.7012 - val_loss: 20197.8301 - val_mean_squared_error: 20197.8301
Epoch 4/2000
3/3 [==============================] - 0s 22ms/step - loss: 14201.2285 - mean_squared_error: 14201.2285 - val_loss: 20195.4863 - val_mean_squared_error: 20195.4863
Epoch 5/2000
3/3 [==============================] - 0s 23ms/step - loss: 14190.2275 - mean_squared_error: 14190.2275 - val_loss: 20193.0078 - val_mean_squared_error: 20193.0078
Epoch 6/2000
3/3 [==============================] - 0s 23ms/step - loss: 14157.4277 - mean_squared_error: 14157.42

[361.7874450683594, 361.7874450683594]

In [99]:
model.save('model.h5')